幅広くデータを見ていく
* 加速度の波形を並べていみる
* ウィンドウフレームごとの平均，分散の変化

In [1]:
# 表示用
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

In [2]:
import numpy as np
from numpy import zeros, newaxis
import datetime
import os
import copy
import time

# Parametor about Sensor
SensorNum=1 # Sensor number
#SensorAxis='AccX' # Axis

# Parametor about  " Sliding Window " 
WindowWidth =np.array([128,256,512,1024,2048,4096]) # Window Width
SlidingWidth =WindowWidth/4 #sliding window

# Parametor about Neural Network
AEDimention = 16

# Define Data Name
DATE= str( datetime.date.today() )+'/'
TITLE="Participants01_First_1/"

# Parametor about " Data Path"
DataPath = "/media/takeyama/Transfer/02_ActivityResearch/Paticipants01/01_First/20161018-105301/mem/"

#chopin
#StorePath = "/home/takeyama/Documents/"
#mozart
StorePath ="/media/takeyama/HD-PNFU3/01_ActivityResearchData/"
if not os.path.exists(StorePath+TITLE+'dictionary/'): 
    os.makedirs(StorePath+TITLE+'dictionary/')
DictionaryDataPath=StorePath+TITLE+'dictionary/'

if not os.path.exists(StorePath+TITLE+'window/'): 
    os.makedirs(StorePath+TITLE+'window/')
WindowDataPath=StorePath+TITLE+'window/'

if not os.path.exists(StorePath+TITLE+DATE+'studyOutput/'): 
    os.makedirs(StorePath+TITLE+DATE+'studyOutput/')
StudyOutputPath=StorePath+TITLE+DATE+'studyOutput/'

if not os.path.exists(StorePath+TITLE+DATE+'parametor/'): 
    os.makedirs(StorePath+TITLE+DATE+'parametor/')
ParametorPath=StorePath+TITLE+DATE+'parametor/'

if not os.path.exists(StorePath+TITLE+DATE+'result/'): 
    os.makedirs(StorePath+TITLE+DATE+'result/')
ResultPath=StorePath+TITLE+DATE+'result/'

if not os.path.exists(StorePath+'/'+TITLE+DATE+'graph/'): 
    os.makedirs(StorePath+'/'+TITLE+DATE+'graph/')
GraphPath=StorePath+'/'+TITLE+DATE+'graph/'

if not os.path.exists(StorePath+'/'+TITLE+'feature/'): 
    os.makedirs(StorePath+'/'+TITLE+'feature/')
FeaturePath=StorePath+'/'+TITLE+'feature/'

センサデータの読み込み

In [3]:
# センサデータのcsvファイル名をリストに集約する関数
def SensorDataFileNameList(path):
    l = os.listdir(DataPath)
    FilePath =[]
    SensorName = []
    
    for n in range(len(l)):
        if (l[n][:4].find('mem-') != -1):
            FilePath.append(l[n])
            SensorName.append(l[n][4:15])
    
    return FilePath,SensorName

CSVを読み込んで，CSVの中身を辞書型で表現する

In [4]:
# FFT library
from scipy import fftpack
from scipy import signal

# Graph Plot
import matplotlib.pyplot as plt
from matplotlib import gridspec

# Generate WindowFrame Function
from mymodule import window

# timer
import time

# temp file 
import tempfile

%matplotlib inline

In [5]:
DictName = os.listdir(DictionaryDataPath)
print DictName

['Chest.npz', 'LeftHand.npz', 'LeftLeg.npz', 'RightHand.npz', 'Rightleg.npz', 'West.npz']


In [6]:
Sensor5 = np.load(DictionaryDataPath+DictName[0])['data'][()]
Sensor1 = np.load(DictionaryDataPath+DictName[1])['data'][()]
Sensor3 = np.load(DictionaryDataPath+DictName[2])['data'][()]
Sensor2 = np.load(DictionaryDataPath+DictName[3])['data'][()]
Sensor4 = np.load(DictionaryDataPath+DictName[4])['data'][()]
Sensor6 = np.load(DictionaryDataPath+DictName[5])['data'][()]

In [7]:
def GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH):
    """
    data　ウィンドウフレームに変換するデータ
    registName　ウィンドウフレームに登録するデータの名前
    windowWidth　ウィンドウ幅
    slidingWidth　スライド幅
    PATH ウィドウフレームを保存するディレクトリ
    
    
    About Function:
        与えられたdataからウィンドウ幅windowWidth,スライド幅slidingWidthにしたがって
        registNameのウィドウフレームを返す．
        また，与えれたPATH内に同様なパラメータ( WindowWidth, slidingWidth)かつ同様な
        windowNameのものがある場合，そのデータを返す．
        この関数が登録，ウィンドウフレームに変換できるデータは１つとする
    """
    storedName = windowName+'_Win='+str(wWidth).zfill(4)+'_Sld='+str(sWidth).zfill(4)+'.npz'
    l = os.listdir(PATH)
    
    if storedName in l:
        print "this data had finished making"
        return np.load(PATH+storedName)['data'][()]
    
    w=window()
    w.SetData(windowName,dataRaw)    
    wind=w.Compile(wWidth,sWidth)
    windoW=wind.reshape((len(wind),np.prod(wind.shape[1:])))
    
    np.savez(PATH+storedName,data=wind)

    return wind

In [8]:
windowNum = 256
slidingNum = windowNum/32

SaveFileName=Sensor1['Name']+'_AccX'+str(windowNum).zfill(4)+str(slidingNum).zfill(4)

wind=GetWindowFrame('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)

this data had finished making


# Methematical/Statistical Functions

## Mean, Median

In [9]:
def Vector_Mean(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    
    Output = np.array([])
    for i in tqdm_notebook( range( wind.shape[0] ),desc='Mean' ):
        Output = np.append(Output, wind[i,:].mean())
    return Output

In [10]:
def Vector_Median(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    
    Output = np.array([])
    for i in tqdm_notebook( range( wind.shape[0] ) ,desc='Median'):
        Output = np.append(Output, np.median(wind[i,:].mean()) )
    return Output

## Variance, Std Deviation

In [11]:
def Vector_Var(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    
    Output = np.array([])
    for i in tqdm_notebook( range( wind.shape[0] ) ,desc='Var' ):
        Output = np.append(Output, wind[i,:].var())
    return Output

In [12]:
def Vector_StdDev(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    
    Output = np.array([])
    for i in tqdm_notebook( range( wind.shape[0] ),desc='StdDev' ):
        Output = np.append(Output, wind[i,:].var()**0.5)
    return Output

## Min, Max, Range
Range = the difference between maximum and minimum sample values

In [13]:
def Vector_Max(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    
    Output = np.array([])
    for i in tqdm_notebook( range( wind.shape[0] ) ,desc='Max'):
        Output = np.append(Output, wind[i,:].max())
    return Output

In [14]:
def Vector_Min(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    
    Output = np.array([])
    for i in tqdm_notebook( range( wind.shape[0] ) ,desc='Min'):
        Output = np.append(Output, wind[i,:].min())
    return Output

In [15]:
def Vector_Range(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    
    Output = np.array([])
    for i in tqdm_notebook( range( wind.shape[0] ) ,desc='Range'):
        Output = np.append(Output, wind[i,:].max()-wind[i,:].min())
    return Output

## RMS

In [16]:
def Vector_RMS(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    
    Output = np.array([])
    for i in tqdm_notebook( range( wind.shape[0] ) ,desc='RMS'):
        Output = np.append(Output, (np.sum( wind[i,:]**2)/wind[i,:].size)**0.5 )
    return Output

## Correlation, Cross-Correlation

def Vector_Corr(windowNameX, dataRawX, windowNameY, dataRawY, wWidth, sWidth, PATH):
    # get windowFrame
    windX=GetWindowFrame(windowNameX, dataRawX, wWidth, sWidth, PATH)
    windY=GetWindowFrame(windowNameY, dataRawY, wWidth, sWidth, PATH)

    Output = np.array([])
    tmpArray = np.array([])
    
    for i in tqdm_notebook( range( windX.shape[0] ) ):
        tmpArray = np.array([])
        for l in tqdm_notebook( range(windY.shape[0]),leave=False):
            tmpArray = np.append(tmpArray, np.corrcoef(windX[i,:],windY[l,:]) ) 
            
        Output = np.append(Output, tmpArray, axis=0)
    return Output  

In [17]:
def Vector_Corr(windowNameX, dataRawX, windowNameY, dataRawY, wWidth, sWidth, PATH):
    # get windowFrame
    windX=GetWindowFrame(windowNameX, dataRawX, wWidth, sWidth, PATH)
    windY=GetWindowFrame(windowNameY, dataRawY, wWidth, sWidth, PATH)

    Output = np.array([])
    tmpArray = np.array([])
    
    # devine num
    dev = 10
    
    for i in tqdm_notebook( range(0, windX.shape[0], windX.shape[0]/dev) ,desc='Correlation'):
        for d in range(windX.shape[0]/10):
            tmpArray = np.array([])
            for l in tqdm_notebook( range(windY.shape[0]),leave=False):
                tmpArray = np.append(tmpArray, np.corrcoef(windX[i+d,:],windY[l,:]) ) 

            Output = np.append(Output, tmpArray, axis=0)
        np.savez('tmpfile_'+str( int(i/windX.shape[0]) ),data=Output)
        Output = np.array([])
    
    for i in range(dev):
        OutputSum = np.append(OutputSum, np.load('tmpfile_'+str(i)+'.npz')['data'][()])
    return OutputSum

# Signal magnitude area

In [ ]:
def Vector_SMA(windowName, dataRawX, dataRawY, dataRawZ, wWidth, sWidth, PATH):
    # get windowFrame
    windX=GetWindowFrame(windowName, dataRawX, wWidth, sWidth, PATH)
    windY=GetWindowFrame(windowName, dataRawY, wWidth, sWidth, PATH)
    windZ=GetWindowFrame(windowName, dataRawZ, wWidth, sWidth, PATH)
    
    # windowFrame roop
    for i in tqdm_notebook( range( windX.shape[0] ) ,desc='SMA'):
        SqrMag = windX[i,:]**2+windY[i,:]**2+windZ[i,:]**2
        Mag = np.sum( SqrMag**0.5 )

        Output = np.append(Output, Mag)
    return Output  

# データからベクトルを算出する

In [ ]:
windowNum = 256
slidingNum = windowNum/32

SaveFileName=Sensor1['Name']+'_AccX'+'_Win='+str(windowNum).zfill(4)+'_Sld='+str(slidingNum).zfill(4)

wind=GetWindowFrame('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)

np.savez(FeaturePath+SaveFileName+'_Mean',data=\
     Vector_Mean('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)
)
np.savez(FeaturePath+SaveFileName+'_Median',data=\
    Vector_Median('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)
)
np.savez(FeaturePath+SaveFileName+'_Median',data=\
     Vector_Var('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)
)
np.savez(FeaturePath+SaveFileName+'_Median',data=\
     Vector_StdDev('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)
)
np.savez(FeaturePath+SaveFileName+'_Median',data=\
     Vector_Max('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)
)
np.savez(FeaturePath+SaveFileName+'_Median',data=\
    Vector_Min('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)
)
np.savez(FeaturePath+SaveFileName+'_Median',data=\
    Vector_Range('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)
)
np.savez(FeaturePath+SaveFileName+'_Median',data=\
     Vector_RMS('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)
)
np.savez(FeaturePath+SaveFileName+'_Median',data=\
    Vector_Corr('AccX', Sensor1['AccX'],'AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)
)
np.savez(FeaturePath+SaveFileName+'_Median',data=\
    Vector_Corr('AccX', Sensor1['AccX'],'AccY', Sensor1['AccY'], windowNum, slidingNum, WindowDataPath)
)
np.savez(FeaturePath+SaveFileName+'_Median',data=\
    Vector_SMA('AccXYZ', Sensor1['AccX'], Sensor1['AccY'], Sensor1['AccZ'], windowNum, slidingNum, WindowDataPath)
)



this data had finished making
this data had finished making

this data had finished making

this data had finished making

this data had finished making

this data had finished making

this data had finished making

this data had finished making

this data had finished making

this data had finished making
this data had finished making
